In [20]:
import torch
import torch_geometric as pyg
import torch.nn as nn
import torch_geometric.nn as gnn
from torch_geometric.datasets import Amazon
from torch_geometric.utils import train_test_split_edges, add_self_loops

In [3]:
amazon = Amazon('amazon', 'Computers')
data = amazon[0]

c:\Users\szige\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) 

In [4]:
train_test_split_edges(data)

c:\Users\szige\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[13752, 767], y=[13752], val_pos_edge_index=[2, 12293], test_pos_edge_index=[2, 24586], train_pos_edge_index=[2, 417964], train_neg_adj_mask=[13752, 13752], val_neg_edge_index=[2, 12293], test_neg_edge_index=[2, 24586])

In [5]:
data.train_pos_edge_index

tensor([[    0,     0,     0,  ..., 13751, 13751, 13751],
        [  507,  6551,  8210,  ..., 12479, 12751, 13019]])

In [6]:
features = torch.tensor([
    [1, 1, 1],
    [2, 2, 2],
    [3, 3, 3]
], dtype=torch.float)

edge_index = torch.tensor([
    [0, 1, 2],
    [2, 0, 1]
])

In [7]:
class GINLayer(gnn.MessagePassing):
    def __init__(self, in_features: int, hidden_features: int, out_features: int, dropout: float) -> None:
        super(GINLayer, self).__init__(aggr='add')
        self.in_linear = nn.Linear(in_features, hidden_features)
        self.out_linear = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(out_features)
        self.skip = nn.Linear(in_features, out_features)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        return self.propagate(edge_index, x=x) + self.skip(x)
    
    def message(self, x_j):
        return self.in_linear(x_j)
    
    def update(self, aggr_out: torch.Tensor):
        aggr_out = torch.relu(aggr_out)
        aggr_out = self.dropout(aggr_out)
        aggr_out = self.out_linear(aggr_out)
        aggr_out = self.layernorm(aggr_out)
        return aggr_out


In [28]:
class GINLayer(gnn.MessagePassing):
    def __init__(self, in_features: int, hidden_features: int, out_features: int):
        super(GINLayer, self).__init__(aggr='add')
        self.mlp_f = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features)
        )

        self.mlp_o = nn.Sequential(
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features)
        )
        self.eps = nn.Parameter(torch.rand(size=(1,)))

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.shape[0])
        out = self.propagate(edge_index=edge_index, x=x)
        eps = (1 + self.eps)
        out = eps * self.mlp_f(x) + out
        out = self.mlp_o(out)
        return out

    def message(self, x_j: torch.Tensor):
        return self.mlp_f(x_j)


In [29]:
gin = GINLayer(features.shape[-1], 16, 1)
gin(features, edge_index).shape

torch.Size([3, 1])

In [ ]:
class MyGNN(nn.Module):
    def __init__(self, in_features: int, hidden_features: int, out_features: int, num_layers: int, dropout: float):
        super(MyGNN, self).__init__()
        self.dropout = dropout
        self.layers = nn.ModuleList([])
        self.layers.append(GINLayer(in_features, hidden_features, hidden_features, self.dropout))
        for _ in range(num_layers - 1):
            self.layers.append(GINLayer(hidden_features, hidden_features, hidden_features, self.dropout))

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x

In [ ]:
in_features = features.shape[-1]
hidden_features = 8
out_features = 1
layer = GINLayer(in_features, hidden_features, out_features, 0.2)
layer(features, edge_index)

tensor([[0.7909],
        [1.2253],
        [1.6598]], grad_fn=<AddBackward0>)

In [23]:
model = MyGNN(in_features, hidden_features, out_features, 4, dropout=0.2)
model(features, edge_index)

tensor([[ 0.2428,  0.8061, -2.6719, -0.1753,  0.6900, -0.0668, -2.8138,  2.0833],
        [-0.4139,  1.4587, -1.3732, -0.9616,  1.4975,  0.1412, -1.4935,  1.0271],
        [-0.9229,  1.4200, -1.8686, -0.8225,  0.6138,  0.2466, -2.0225,  0.7368]],
       grad_fn=<AddBackward0>)

In [35]:
src, dst = edge_index
torch.cat([features[src], features[dst]], dim=-1)

tensor([[1., 1., 1., 3., 3., 3.],
        [2., 2., 2., 1., 1., 1.],
        [3., 3., 3., 2., 2., 2.]])